# 분석 결과 비교: 가정 시나리오별 신뢰도 추정

**연구 목표:** '낙관적' 가정과 '보수적' 가정이라는 두 가지 상이한 시나리오 하에서 계층적 베이즈 모델을 각각 실행하고, 그 결과를 비교하여 가정의 변화가 최종 신뢰도 추정치에 미치는 영향을 시각적으로 분석한다.

**분석 과정:**
1. 두 가지 시나리오('낙관적', '보수적')를 정의한다.
2. 각 시나리오에 대해 PyMC 모델을 실행하고 MCMC 샘플링 결과를 저장한다.
3. **시각화 1:** 롯트별 신뢰도 추정 결과를 포레스트 플롯으로 나란히 비교한다.
4. **시각화 2:** 전체 비축물량의 평균 신뢰도 분포를 하나의 그래프에 겹쳐서 비교한다.

## 1. 환경 설정 및 데이터 준비

In [ ]:
import pymc as pm
import pandas as pd
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 스타일 설정
sns.set_theme(style="whitegrid")
try:
    plt.rcParams['font.family'] = 'Malgun Gothic' # Windows
except:
    plt.rcParams['font.family'] = 'AppleGothic' # Mac
plt.rcParams['axes.unicode_minus'] = False

# 가상 데이터 생성
data = pd.DataFrame({
    'production_lot': ['2015-LOT-01', '2015-LOT-02', '2015-LOT-04'],
    'num_tested': [11, 10, 12],
    'num_failures': [0, 1, 1]
})
data['num_success'] = data['num_tested'] - data['num_failures']

# 모델링을 위한 인덱스 설정
all_years = np.arange(2015, 2020)
all_lots = [f"{year}-LOT-{i:02d}" for year in all_years for i in range(1, 7)]
lot_map = {lot: i for i, lot in enumerate(all_lots)}
year_map = {year: i for i, year in enumerate(all_years)}
year_of_lot = np.array([int(lot.split('-')[0]) for lot in all_lots])
year_idx_of_lot = np.array([year_map[y] for y in year_of_lot])
observed_lot_idx = [lot_map[lot] for lot in data['production_lot']]

## 2. 시나리오별 모델 실행

두 가지 시나리오를 정의하고, 각각에 대해 모델을 실행하여 결과를 `traces` 딕셔너리에 저장합니다.

In [ ]:
# --- 시나리오 정의 ---
scenarios = {
    "낙관적 (Optimistic)": {
        "inter_year_sigma": 0.01, # 연도별 차이가 거의 없음
        "intra_lot_sigma": 0.02,  # 롯트별 차이도 거의 없음
        "degradation_effect_on_variance": False # 시간에 따른 분산 변화 없음
    },
    "보수적 (Pessimistic)": {
        "inter_year_sigma": 0.2,  # 연도별 차이가 클 수 있음
        "intra_lot_sigma": 0.1,   # 롯트별 차이도 클 수 있음
        "degradation_effect_on_variance": True # 시간에 따라 분산 증가
    }
}

traces = {} # 각 시나리오의 분석 결과를 저장할 딕셔너리

# --- 각 시나리오에 대해 모델 실행 ---
for name, params in scenarios.items():
    print(f"\n--- [{name}] 시나리오 분석 실행... ---")
    with pm.Model() as model:
        # Priors
        mu_global_logit = pm.Normal('mu_global_logit', mu=3.89, sigma=0.5)
        sigma_year = pm.HalfNormal('sigma_year', sigma=params["inter_year_sigma"])
        sigma_lot_base = pm.HalfNormal('sigma_lot_base', sigma=params["intra_lot_sigma"])

        if params["degradation_effect_on_variance"]:
            variance_degradation_rate = pm.HalfNormal('variance_degradation_rate', sigma=0.05)
            age_of_lot = 2025 - year_of_lot
            sigma_lot_effective = pm.Deterministic('sigma_lot_effective', sigma_lot_base + age_of_lot * variance_degradation_rate)
        else:
            sigma_lot_effective = pm.Deterministic('sigma_lot_effective', sigma_lot_base)

        # Hierarchy
        theta_year = pm.Normal('theta_year', mu=mu_global_logit, sigma=sigma_year, shape=len(all_years))
        theta_lot = pm.Normal('theta_lot', mu=theta_year[year_idx_of_lot], sigma=sigma_lot_effective, shape=len(all_lots))
        reliability_lot = pm.Deterministic('reliability_lot', pm.invlogit(theta_lot))

        # Likelihood
        y_obs = pm.Binomial(
            'y_obs',
            n=data['num_tested'].values,
            p=reliability_lot[observed_lot_idx],
            observed=data['num_success'].values
        )

        # Sampling
        traces[name] = pm.sample(2000, tune=1500, cores=1, return_inferencedata=True, random_seed=2024, progressbar=True)
    print(f"--- [{name}] 시나리오 분석 완료. ---")

## 3. 결과 비교 시각화

### 결과물 예시 1: 시나리오별 포레스트 플롯(Forest Plot) 비교

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 12), sharey=True)

for i, (name, trace) in enumerate(traces.items()):
    ax = axes[i]
    az.plot_forest(
        trace, 
        var_names=['reliability_lot'], 
        combined=True, 
        hdi_prob=0.9, # 90% 신뢰구간(HDI)
        ax=ax
    )
    ax.set_title(f'시나리오: {name}', fontsize=16)
    # y축 레이블은 첫 번째 그래프에만 표시
    if i == 0:
        ax.set_yticklabels(all_lots[::-1])
    else:
        ax.set_yticklabels([])

fig.suptitle('가정 시나리오별 롯트 신뢰도 추정 비교 (90% HDI)', fontsize=20, y=0.95)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### 결과물 예시 2: 전체 비축물량 신뢰도 확률 분포 비교

In [ ]:
plt.figure(figsize=(12, 7))

for name, trace in traces.items():
    # 전체 비축물량의 평균 신뢰도 계산
    posterior_samples = trace.posterior['reliability_lot'].values
    mean_stockpile_reliability = posterior_samples.mean(axis=2) # 롯트 축에 대해 평균
    
    # 분포 시각화
    sns.kdeplot(mean_stockpile_reliability.flatten(), label=f'시나리오: {name}', fill=True, alpha=0.5)
    
    # 90% 신뢰구간(HDI) 하한값 계산 및 출력
    hdi_lower_bound = az.hdi(mean_stockpile_reliability, hdi_prob=0.9)[0]
    print(f"[{name} 시나리오] 90% 신뢰수준에서 전체 비축물량의 평균 신뢰도는 최소 {hdi_lower_bound:.3f} 이상일 것으로 추정됩니다.")

plt.title('가정 시나리오별 전체 비축물량 평균 신뢰도 분포 비교', fontsize=18)
plt.xlabel('전체 평균 신뢰도', fontsize=12)
plt.ylabel('확률 밀도', fontsize=12)
plt.legend(fontsize=12)
plt.show()